In [96]:
import pandas as pd
from sqlalchemy import create_engine, text
!pip install mysqlclient

In [97]:
import pandas as pd

# Specify the path to your CSV file
file_path = '/content/stockdata.csv'  # Replace this with the actual path to your file

# Read the CSV file into a DataFrame
source_csv_df = pd.read_csv(file_path)

# Display the DataFrame
print(source_csv_df)


             date     open      high       low   close    volume ticker  \
0      2024-04-26  412.170  413.0000  405.7601  406.32  29694654   MSFT   
1      2024-04-25  394.030  399.8899  388.0300  399.04  40586450   MSFT   
2      2024-04-24  409.560  412.4700  406.7800  409.06  15065330   MSFT   
3      2024-04-23  404.235  408.2000  403.0600  407.57  15734501   MSFT   
4      2024-04-22  400.080  402.8500  395.7500  400.96  20286875   MSFT   
...           ...      ...       ...       ...     ...       ...    ...   
28345  1999-11-05   35.250   36.1200   35.1200   35.50   2344900    TSM   
28346  1999-11-04   35.440   35.7500   34.7500   35.00   1109100    TSM   
28347  1999-11-03   35.000   35.4400   34.8700   35.44   1577400    TSM   
28348  1999-11-02   35.000   35.5000   34.8700   35.25   2467600    TSM   
28349  1999-11-01   34.370   35.2500   33.5000   34.06   3106500    TSM   

               company_name  
0                 Microsoft  
1                 Microsoft  
2        

In [88]:
raw_host = "sqlproject.c5acwaqw021i.us-east-2.rds.amazonaws.com"
raw_username = "jonahwoo"
raw_password = "isba4715"
raw_schema = "sql_project"

raw_db_config = {
    "host": raw_host,
    "username": raw_username,
    "password": raw_password,
    "schema": raw_schema
}

raw_engine = create_engine(f"mysql+mysqldb://{raw_db_config['username']}:{raw_db_config['password']}@{raw_db_config['host']}/{raw_db_config['schema']}")

In [89]:
raw_table = "raw_Stock_Data"
source_csv_df.to_sql(raw_table, raw_engine, index=False, if_exists="append")

28350

## DESCRIPTIVE ANALYTICS BUSINESS QUESTION

What is each stock's 50 day moving average?

In [102]:
## sql to answer the question
raw_query = text(f'''
SELECT
    ticker,
    DATE,
    AVG(close) OVER (PARTITION BY ticker ORDER BY DATE ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) AS fifty_day_moving_average
FROM
    {raw_table}
ORDER BY
    DATE DESC;

''')

raw_df = pd.read_sql(raw_query, raw_engine)

In [103]:
## sql results
raw_df

,ticker,DATE,fifty_day_moving_average
0,TSM,2024-04-26,134.9970
1,TSM,2024-04-26,135.0814
2,TSM,2024-04-26,135.1658
3,TSM,2024-04-26,135.2502
4,TSM,2024-04-26,135.3346
...,...,...,...
141745,AAPL,1999-11-01,77.6200
141746,AAPL,1999-11-01,77.6200
141747,AAPL,1999-11-01,77.6200
141748,AAPL,1999-11-01,77.6200


## INSIGHTS



All of the current closing prices are higher than their 50 day moving average except for MSFT.

## RECOMMENDATION

As most of the prices are above their average, I would sit back and wait for a drop in the market.

## PREDICTION

The closing prices will continue to be higher than the fifty day moving average for a while.

## DIAGNOSTIC ANALYTICS BUSINESS QUESTION

What do the previous query results mean?

In [100]:
## sql code

raw_query = text(f'''
WITH moving_average_cte AS (
    SELECT
        ticker,
        DATE,
        AVG(close) OVER (PARTITION BY ticker ORDER BY DATE ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) AS fifty_day_moving_average,
        close AS closing_price,
        ROW_NUMBER() OVER (PARTITION BY ticker ORDER BY DATE DESC) AS rn
    FROM
        {raw_table}
)
SELECT
    ticker,
    DATE,
    fifty_day_moving_average,
    closing_price,
    CASE
        WHEN closing_price > fifty_day_moving_average THEN 'Overvalued'
        WHEN closing_price < fifty_day_moving_average THEN 'Undervalued'
        ELSE 'Equal to Average'
    END AS valuation_status
FROM
    moving_average_cte
WHERE
    rn = 1
GROUP BY
    ticker;

''')

raw_df = pd.read_sql(raw_query, raw_engine)

In [101]:
## sql results

raw_df

,ticker,DATE,fifty_day_moving_average,closing_price,valuation_status
0,AAPL,2024-04-26,168.7410,169.30,Overvalued
1,AVGO,2024-04-26,1275.5040,1344.07,Overvalued
2,MSFT,2024-04-26,407.5748,406.32,Undervalued
3,NVDA,2024-04-26,830.5776,877.35,Overvalued
4,TSM,2024-04-26,134.9970,138.30,Overvalued


## INSIGHTS

Most of the stocks from my dataset are overvalued.

## RECOMMENDATIONS

Wait for the stocks to either be correctly valued or undervalued before buying.

## PREDICTION

None of the stocks will be consistently undervalued for a while.